In [2]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
from datetime import date
import os
from dotenv import load_dotenv


scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_file("credentials.json", scopes=scopes)
client = gspread.authorize(creds)

load_dotenv()
spreadheet_key = os.getenv('SPREADSHEET_KEY')
sheet = client.open_by_key(spreadheet_key).sheet1

In [3]:

data = sheet.get_all_records()
df = pd.DataFrame(data)

In [4]:
isinstance(df["Product"], str)
# df["Product"]
# df["error_flags"] = df.apply(funkcja spr, axis=1)

False

In [5]:
s = pd.Series(data)

In [6]:
error = []
if not isinstance(df.iloc[0]["OrderID"], int):
    error.append("OrderID not int")
else:
    print()

In [7]:
from datetime import date

RULES = {
    "OrderID": {
        "required": True,
        "type": int,
        "min": 1
    },
    "Product": {
        "required": True,
        "type": str,
    },
    "Quantity": {
        "required": True,
        "type": int,
        "min": 1
    },
    "Price": {
        "required": True,
        "type": (int, float),
        "min": 0
    },
    "Date": {
        "required": True,
        "type": date
    }
}

In [26]:

for column, rules in RULES.items():
    if rules["type"] in [int, float]:
        df[column] = pd.to_numeric(df[column], errors="coerce")
    elif rules["type"] == "date":
        df[column] = pd.to_datetime(df[column], errors="coerce")

all_errors = []


for i, row in df.iterrows():
    errors = []
    for column, rules in RULES.items():
        print(f"{i+1} iteracja:")
        # print(row[column])
        # print(rules["required"])

        if rules["required"] and pd.isnull(row[column]):
            errors.append(f"{column}: Error cell empty")

        if rules["type"] in [int, float]:
            if not pd.api.types.is_numeric_dtype(type(row[column])):
                errors.append(f"{column}: Error data type")

        if "min" in rules and not pd.isnull(row[column]) and row[column] < rules["min"]:
            errors.append(f"{column}: Value error")

    print(f"Row {i} errors: {errors}")
    all_errors.append("; ".join(errors))
df["error_flags"] = all_errors


1 iteracja:
1 iteracja:
1 iteracja:
1 iteracja:
1 iteracja:
Row 0 errors: ['OrderID: Error cell empty']
2 iteracja:
2 iteracja:
2 iteracja:
2 iteracja:
2 iteracja:
Row 1 errors: []
3 iteracja:
3 iteracja:
3 iteracja:
3 iteracja:
3 iteracja:
Row 2 errors: []
4 iteracja:
4 iteracja:
4 iteracja:
4 iteracja:
4 iteracja:
Row 3 errors: []
5 iteracja:
5 iteracja:
5 iteracja:
5 iteracja:
5 iteracja:
Row 4 errors: []
6 iteracja:
6 iteracja:
6 iteracja:
6 iteracja:
6 iteracja:
Row 5 errors: []
7 iteracja:
7 iteracja:
7 iteracja:
7 iteracja:
7 iteracja:
Row 6 errors: []
8 iteracja:
8 iteracja:
8 iteracja:
8 iteracja:
8 iteracja:
Row 7 errors: []
9 iteracja:
9 iteracja:
9 iteracja:
9 iteracja:
9 iteracja:
Row 8 errors: []
10 iteracja:
10 iteracja:
10 iteracja:
10 iteracja:
10 iteracja:
Row 9 errors: []
11 iteracja:
11 iteracja:
11 iteracja:
11 iteracja:
11 iteracja:
Row 10 errors: []
12 iteracja:
12 iteracja:
12 iteracja:
12 iteracja:
12 iteracja:
Row 11 errors: []
13 iteracja:
13 iteracja:
13 ite

In [17]:

df.head()

,OrderID,CustomerName,Product,Quantity,Price,Date,error_flags,Total
0,NaN,Michał Nowicki,Tablet,1,2000,2025-08-06,OrderID: Error cell empty,2000
1,1002.0,Katarzyna Krawczyk,Webcam,4,1200,2025-08-23,,4800
2,1003.0,Paweł Maj,Webcam,2,600,2025-08-12,,1200
3,1004.0,Paweł Maj,Headphones,3,1200,2025-08-21,,3600
4,1005.0,Piotr Wiśniewski,Docking Station,3,1800,2025-08-26,,5400


In [29]:

df["Total cost"] =df["Quantity"] * df["Price"]
df = df.fillna("")

if pd.api.types.is_datetime64_any_dtype(df["Date"]):
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

sheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1u17H40MV8MLdAJKaZqu6WbLxb7d8_CU8M6werqpS6WM',
 'updatedRange': 'orders!A1:H51',
 'updatedRows': 51,
 'updatedColumns': 8,
 'updatedCells': 408}

In [11]:
df.head()

,OrderID,CustomerName,Product,Quantity,Price,Date,error_flags,Total
0,NaN,Michał Nowicki,Tablet,1,2000,2025-08-06,OrderID: Error cell empty,2000
1,1002.0,Katarzyna Krawczyk,Webcam,4,1200,2025-08-23,,4800
2,1003.0,Paweł Maj,Webcam,2,600,2025-08-12,,1200
3,1004.0,Paweł Maj,Headphones,3,1200,2025-08-21,,3600
4,1005.0,Piotr Wiśniewski,Docking Station,3,1800,2025-08-26,,5400


In [12]:
# Sprzedaz dzienna

daily_sales = df.groupby("Date")["Total"].sum()
print(daily_sales)

Date
2025-08-01    70900
2025-08-02    15200
2025-08-03     2600
2025-08-06     4000
2025-08-09    50000
2025-08-10    15000
2025-08-11     4200
2025-08-12     2280
2025-08-13     8000
2025-08-14    11100
2025-08-15     1000
2025-08-16    51080
2025-08-19    15080
2025-08-20     1080
2025-08-21    30000
2025-08-22     3600
2025-08-23    14400
2025-08-24    71120
2025-08-25    50000
2025-08-26    69800
2025-08-27    11600
2025-08-28     1200
2025-08-29    20160
Name: Total, dtype: int64


In [68]:
from datetime import datetime, timedelta

def sales_analysis(time_period=None, start_date=None, end_date=None):
    if time_period is not None and (start_date is not None or end_date is not None):
        raise ValueError("Podaj albo time_period, albo start_date i end_date, nie oba naraz")
    df["Date"] = pd.to_datetime(df["Date"])

    if time_period:
        # time_period = 30
        start_date = datetime.now() - timedelta(time_period)
        sales = df[df["Date"] >= start_date].groupby("Date")["Total"].sum()
        return sales
    elif start_date and end_date:
        sales = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)].groupby("Date")["Total"].sum()
        return sales

# sales=sales_analysis(time_period=30)
sales=sales_analysis(start_date="2025-08-06", end_date="2025-08-15")

# print(sales)
# print(sales2)

In [ ]:
import matplotlib.pyplot as plt

sales.plot(kind="line")  # plt.plot(weekly_sales.index, weekly_sales.values)
plt.title(f"Sales Trend - Last {time_period} Days")
plt.xlabel("Date")
plt.ylabel("Total Sales")
plt.show()


In [ ]:
import seaborn as sns

sns.lineplot(x=sales.index, y=sales.values)

plt.title(f"Sales Trend - Last {time_period} Days")
plt.xlabel("Date")
plt.ylabel("Total Sales")
plt.show()


In [69]:
import plotly.express as px

fig = px.line(sales, x=sales.index, y="Total", title=f"Sales Trend - Last {time_period} Days")
fig.show()


In [79]:
def products_performance(time_period=None, start_date=None, end_date=None):
    if time_period and (start_date or end_date):
        raise ValueError("Podaj albo time_period, albo start_date i end_date, nie oba naraz")

    if time_period:
        # time_period = 30
        start_date = datetime.now() - timedelta(time_period)
        products_within_dates = df[df["Date"] >= start_date]
    elif start_date and end_date:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        products_within_dates = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]
    else:
        products_within_dates = df

    top_products = products_within_dates.groupby("Product").agg(
        total_quantity=("Quantity", "sum"),
        num_orders=("OrderID", "count"),
        total_revenue=("Total", "sum")
    )
    top_products = top_products.sort_values(by="total_quantity", ascending=False)
    return top_products

# products_performance = products_performance(start_date="2025-08-06", end_date="2025-08-15")
products_performance = products_performance()
print(products_performance)

                 total_quantity  num_orders  total_revenue
Product                                                   
Docking Station              29           9          66600
Mouse                        27           9          11400
Webcam                       24           8          24600
Tablet                       21           7         170000
Laptop                       17           6         199500
Monitor                      15           6          40500
Headphones                    8           3           8800
Keyboard                      4           2           2000


In [83]:

top_customers = df.groupby("CustomerName").agg(
    num_orders = ("OrderID", "count"),
    orders_cost = ("Total", "sum"),
    last_order = ("Date", "max")
)

top_customers["days_since_last_order"] = (datetime.now() - pd.to_datetime(top_customers["last_order"])).dt.days
top_customers = top_customers.drop(columns="last_order").sort_values(by="num_orders", ascending=False)

print(top_customers)

                    num_orders  orders_cost  days_since_last_order
CustomerName                                                      
Anna Kowalska                8        77200                      7
Michał Nowicki               8       154120                      5
Katarzyna Krawczyk           7        94000                     10
Paweł Maj                    7        12260                     13
Maria Wiśniewska             5        12280                      5
Tomasz Lis                   5        73060                      5
Ewa Zielińska                4        22880                      5
Piotr Wiśniewski             3        22800                      8
Joanna Dąbrowska             2        51200                      6
Jan Nowak                    1         3600                     32
